# PyOPE 使用演示

这个 notebook 演示了 PyOPE 库的主要功能，用于计算顶点算符代数（VOA）中的算符乘积展开（OPE）。

## 1. 导入库

In [38]:
import sys
sys.path.insert(0, '/Users/lelouch/Nutstore Files/Math and Physics/NoteBooks/opepackage/pyope')

from pyope import (
    BasisOperator,
    OPE,
    NO,
    d,
    dn,
    bracket,
    One,
    Delta,
)
from pyope.ope_data import OPEData
import sympy as sp

# 初始化 SymPy 打印
sp.init_printing(use_unicode=True)

## 2. 创建基础算符

创建玻色算符和费米算符：

In [39]:
# 玻色算符
T = BasisOperator("T", bosonic=True)  # 能动张量
J = BasisOperator("J", bosonic=True)  # 流算符

# 费米算符
psi = BasisOperator("ψ", bosonic=False)

print(f"T 是玻色算符: {T.is_bosonic}")
print(f"ψ 是费米算符: {psi.is_fermionic}")
print(f"T 的宇称: {T.parity}")
print(f"ψ 的宇称: {psi.parity}")

T 是玻色算符: True
ψ 是费米算符: True
T 的宇称: 0
ψ 的宇称: 1


## 3. 定义 OPE

定义 Virasoro 代数的 OPE：$T(z)T(w) \sim \frac{c/2}{(z-w)^4} + \frac{2T(w)}{(z-w)^2} + \frac{\partial T(w)}{z-w}$

In [40]:
# 定义中心荷
c = sp.Symbol('c')

# 定义 T(z)T(w) 的 OPE
OPE[T, T] = OPE.make([
    c/2 * One,  # 4阶极点
    0,          # 3阶极点
    2*T,        # 2阶极点
    d(T)        # 1阶极点
])

print("T(z)T(w) 的 OPE:")
display(OPE(T, T))
OPE(T, T).display()

d(OPE(T,d(T)).pole(2))

T(z)T(w) 的 OPE:


OPEData({4: c*One/2, 2: 2*T, 1: ∂T})

<IPython.core.display.Math object>

AttributeError: 'Mul' object has no attribute 'name'

## 4. 访问 OPE 的极点

In [27]:
ope_TT = OPE(T, T)

print(f"最高极点阶数: {ope_TT.max_pole}")
print(f"4阶极点: {ope_TT.pole(4)}")
print(f"3阶极点: {ope_TT.pole(3)}")
print(f"2阶极点: {ope_TT.pole(2)}")
print(f"1阶极点: {ope_TT.pole(1)}")

最高极点阶数: 4
4阶极点: c*One/2
3阶极点: 0
2阶极点: 2*T
1阶极点: ∂T


## 5. OPE 的线性性

测试 OPE 的线性性质：$\text{OPE}(A, B+C) = \text{OPE}(A,B) + \text{OPE}(A,C)$

In [28]:
# 创建另一个算符
K = BasisOperator("K", bosonic=True)

# 定义简单的 OPE
OPE[T, J] = OPEData({1: J})
OPE[T, K] = OPEData({1: K})

# 测试线性性
result = OPE(T, J + K)
expected = OPE(T, J) + OPE(T, K)

print(f"OPE(T, J+K) = {result}")
print(f"OPE(T,J) + OPE(T,K) = {expected}")
print(f"线性性成立: {result == expected}")

OPE(T, J+K) = << 1|| J + K >>
OPE(T,J) + OPE(T,K) = << 1|| J + K >>
线性性成立: True


## 6. 正规序算符

创建正规序乘积 $:AB:(z)$

In [29]:
# 创建正规序算符
NO_TJ = NO(T, J)

print(f"正规序算符: {NO_TJ}")
print(f"左算符: {NO_TJ.left}")
print(f"右算符: {NO_TJ.right}")
print(f"宇称: {NO_TJ.parity}")

正规序算符: NO[T,J]
左算符: T
右算符: J
宇称: 0


## 7. 导数算符

创建算符的导数 $\partial A$, $\partial^2 A$

In [30]:
# 一阶导数
dT = d(T)
print(f"∂T = {dT}")
print(f"导数阶数: {dT.order}")
print(f"基础算符: {dT.base}")

# 二阶导数
d2T = d(T, 2)
print(f"\n∂²T = {d2T}")
print(f"导数阶数: {d2T.order}")

# 使用 dn 函数
d3T = dn(3, T)
print(f"\n∂³T = {d3T}")
print(f"导数阶数: {d3T.order}")

∂T = ∂T
导数阶数: 1
基础算符: T

∂²T = ∂^2T
导数阶数: 2

∂³T = ∂^3T
导数阶数: 3


## 8. 对易子和反对易子

计算对易子 $[A,B] = AB - BA$ 和反对易子 $\{A,B\} = AB + BA$

In [ ]:
# 对易子（玻色算符）
commutator = bracket(T, J, anticommutator=False)
print(f"[T, J] = {commutator}")

# 反对易子（费米算符）
chi = BasisOperator("χ", bosonic=False)
anticommutator = bracket(psi, chi, anticommutator=True)
print(f"{{ψ, χ}} = {anticommutator}")

## 9. 索引算符

创建带索引的算符，如 $J^i$

In [11]:
# 创建可索引的算符
J_indexed = BasisOperator("J", bosonic=True, indexed=True)

# 创建索引实例
i = sp.Symbol('i')
j = sp.Symbol('j')

J_i = J_indexed[i]
J_j = J_indexed[j]

print(f"J[i] = {J_i}")
print(f"J[j] = {J_j}")
print(f"基础名称: {J_i.base_name}")
print(f"索引: {J_i.indices}")

J[i] = J[i]
J[j] = J[j]
基础名称: J
索引: (i,)


## 10. Kronecker Delta 函数

In [12]:
# 数值索引
print(f"δ(1,1) = {Delta(1, 1)}")
print(f"δ(1,2) = {Delta(1, 2)}")

# 符号索引
i, j = sp.symbols('i j')
delta_ij = Delta(i, j)
print(f"δ(i,j) = {delta_ij}")

# 简化
delta_ii = Delta(i, i)
print(f"δ(i,i) = {delta_ii}")

δ(1,1) = 1
δ(1,2) = 0
δ(i,j) = DeltaFunction(i, j)
δ(i,i) = 1


## 11. OPE 数据的算术运算

In [13]:
# 创建两个 OPE
ope1 = OPEData({2: T, 1: J})
ope2 = OPEData({2: J, 1: T})

print(f"OPE1 = {ope1}")
print(f"OPE2 = {ope2}")

# 加法
ope_sum = ope1 + ope2
print(f"\nOPE1 + OPE2 = {ope_sum}")

# 标量乘法
ope_scaled = 2 * ope1
print(f"2 * OPE1 = {ope_scaled}")

# 减法
ope_diff = ope1 - ope2
print(f"OPE1 - OPE2 = {ope_diff}")

OPE1 = << 2|| T 1|| J >>
OPE2 = << 2|| J 1|| T >>

OPE1 + OPE2 = << 2|| J + T 1|| J + T >>
2 * OPE1 = << 2|| 2*T 1|| 2*J >>
OPE1 - OPE2 = << 2|| -J + T 1|| J - T >>


## 12. 完整示例：流代数

定义 Kac-Moody 流代数的 OPE：$J^a(z)J^b(w) \sim \frac{k\delta^{ab}}{(z-w)^2} + \frac{if^{abc}J^c(w)}{z-w}$

In [19]:
# 定义参数
k = sp.Symbol('k')  # 层数
a, b, c = sp.symbols('a b c')
f = sp.Function('f')  # 结构常数

# 创建索引流算符
J_current = BasisOperator("J", bosonic=True, indexed=True)
J_a = J_current[a]
J_b = J_current[b]
J_c = J_current[c]

# 定义 OPE（简化版本，不包含结构常数求和）
OPE[J_a, J_b] = OPEData({
    2: k * Delta(a, b) * One,
    1: sp.I * f(a, b, c) * J_c
})

print("流代数 OPE: J^a(z)J^b(w)")
print(OPE(J_a, J_b))
print(f"\n2阶极点: {OPE(J_a, J_b).pole(2)}")
print(f"1阶极点: {OPE(J_a, J_b).pole(1)}")

流代数 OPE: J^a(z)J^b(w)
<< 2|| k*DeltaFunction(a, b)*One 1|| I*f(a, b, c)*J[c] >>

2阶极点: k*DeltaFunction(a, b)*One
1阶极点: I*f(a, b, c)*J[c]


## 总结

PyOPE 提供了以下核心功能：

1. **算符系统**：玻色/费米算符、导数算符、正规序算符
2. **OPE 定义和计算**：支持线性性、标量乘法
3. **索引算符**：支持带索引的算符如 $J^i$
4. **代数运算**：对易子、反对易子
5. **数据结构**：稀疏存储的 OPE 极点

这些功能为计算顶点算符代数和共形场论中的算符乘积展开提供了强大的工具。

### 13.4 总结

通过上述验证，我们测试了 PyOPE 实现的导数算符 OPE 规则是否符合 Thielemans 论文中的 eq 3.3.1 和 3.3.2：

1. **eq 3.3.1**: $[\partial A, B]_q = -(q-1)[A, B]_{q-1}$ 
   - 在 Virasoro 代数中验证
   - 在 Kac-Moody 流代数中验证

2. **eq 3.3.2**: $[A, \partial B]_q = (q-1)[A, B]_{q-1} + \partial[A, B]_q$
   - 在 Virasoro 代数中验证
   - 在 Kac-Moody 流代数中验证

这些验证确保了 PyOPE 的导数算符处理逻辑与理论公式一致。

In [ ]:
# 计算 OPE[J_a, ∂J_b]
J_a_dJ_b = OPE(J_a, d(J_b))
J_a_J_b = OPE(J_a, J_b)

print("OPE[J^a, ∂J^b]:")
print(J_a_dJ_b)
print()

# 验证 eq 3.3.2 对于流代数
print("验证 eq 3.3.2 (流代数):")
print("=" * 60)

for q in range(1, J_a_dJ_b.max_pole + 1):
    lhs = J_a_dJ_b.pole(q)
    
    term1 = (q - 1) * J_a_J_b.pole(q - 1) if q > 1 else sp.S.Zero
    term2 = d(J_a_J_b.pole(q)) if J_a_J_b.pole(q) != 0 else sp.S.Zero
    rhs = term1 + term2
    
    diff = sp.expand(lhs - rhs)
    
    print(f"q = {q}:")
    print(f"  [J^a, ∂J^b]_{q} = {lhs}")
    print(f"  (q-1)[J^a, J^b]_{{{q-1}}} = {term1}")
    print(f"  ∂[J^a, J^b]_{q} = {term2}")
    print(f"  右边总和 = {rhs}")
    print(f"  差值 = {diff}")
    print(f"  验证: {'✓ 通过' if diff == 0 else '✗ 失败'}")
    print()

In [21]:
# 使用之前定义的流算符 J_a, J_b
# OPE[J_a, J_b] 已经在 cell-24 中定义

# 计算 OPE[∂J_a, J_b]
dJ_a_J_b = OPE(d(J_a), J_b)

print("OPE[∂J^a, J^b]:")
print(dJ_a_J_b)
print()

# 计算 OPE[J_a, J_b]
J_a_J_b = OPE(J_a, J_b)

print("OPE[J^a, J^b]:")
print(J_a_J_b)
print()

# 验证 eq 3.3.1 对于流代数
print("验证 eq 3.3.1 (流代数):")
print("=" * 60)

for q in range(1, dJ_a_J_b.max_pole + 1):
    lhs = dJ_a_J_b.pole(q)
    rhs = -(q - 1) * J_a_J_b.pole(q - 1) if q > 1 else sp.S.Zero
    diff = sp.expand(lhs - rhs)
    
    print(f"q = {q}:")
    print(f"  [∂J^a, J^b]_{q} = {lhs}")
    print(f"  -(q-1)[J^a, J^b]_{{{q-1}}} = {rhs}")
    print(f"  差值 = {diff}")
    print(f"  验证: {'✓ 通过' if diff == 0 else '✗ 失败'}")
    print()

OPE[∂J^a, J^b]:
<< 3|| -2*k*DeltaFunction(a, b)*One 2|| -I*f(a, b, c)*J[c] >>

OPE[J^a, J^b]:
<< 2|| k*DeltaFunction(a, b)*One 1|| I*f(a, b, c)*J[c] >>

验证 eq 3.3.1 (流代数):
q = 1:
  [∂J^a, J^b]_1 = 0
  -(q-1)[J^a, J^b]_{0} = 0
  差值 = 0
  验证: ✓ 通过

q = 2:
  [∂J^a, J^b]_2 = -I*f(a, b, c)*J[c]
  -(q-1)[J^a, J^b]_{1} = -I*f(a, b, c)*J[c]
  差值 = 0
  验证: ✓ 通过

q = 3:
  [∂J^a, J^b]_3 = -2*k*DeltaFunction(a, b)*One
  -(q-1)[J^a, J^b]_{2} = -2*k*DeltaFunction(a, b)*One
  差值 = 0
  验证: ✓ 通过



### 13.3 使用 Virasoro 模型验证

现在使用 Virasoro 来验证相同的关系。

流代数的 OPE 是：$J^a(z)J^b(w) \sim \frac{k\delta^{ab}}{(z-w)^2} + \frac{if^{abc}J^c(w)}{z-w}$

In [ ]:
# 计算 OPE[T, ∂T]
T_dT = OPE(T, d(T))
T_T = OPE(T, T)

print("OPE[T, ∂T]:")
print(T_dT)
print()

# 验证 eq 3.3.2: [T, ∂T]_q = (q-1)[T, T]_{q-1} + ∂[T, T]_q
print("验证 eq 3.3.2:")
print("=" * 60)

# 检查每个极点
for q in range(1, T_dT.max_pole + 1):
    lhs = T_dT.pole(q)  # [T, ∂T]_q
    
    # 计算右边: (q-1)[T, T]_{q-1} + ∂[T, T]_q
    term1 = (q - 1) * T_T.pole(q - 1) if q > 1 else sp.S.Zero
    term2 = d(T_T.pole(q)) if T_T.pole(q) != 0 else sp.S.Zero
    rhs = term1 + term2
    
    # 简化并比较
    diff = sp.expand(lhs - rhs)
    
    print(f"q = {q}:")
    print(f"  [T, ∂T]_{q} = {lhs}")
    print(f"  (q-1)[T, T]_{{{q-1}}} = {term1}")
    print(f"  ∂[T, T]_{q} = {term2}")
    print(f"  右边总和 = {rhs}")
    print(f"  差值 = {diff}")
    print(f"  验证: {'✓ 通过' if diff == 0 else '✗ 失败'}")
    print()

### 13.2 验证 eq 3.3.2: [T, ∂T]_q = (q-1)[T, T]_{q-1} + ∂[T, T]_q

现在计算 $[T, \partial T]_q$ 并验证它是否等于 $(q-1)[T, T]_{q-1} + \partial[T, T]_q$。

In [23]:
# 计算 OPE[∂T, T]
dT_T = OPE(d(T), T)

print("OPE[∂T, T]:")
print(dT_T)
print()

# 计算 OPE[T, T]
T_T = OPE(T, T)

print("OPE[T, T]:")
print(T_T)
print()

# 验证 eq 3.3.1: [∂T, T]_q = -(q-1)[T, T]_{q-1}
print("验证 eq 3.3.1:")
print("=" * 60)

# 检查每个极点
for q in range(1, dT_T.max_pole + 1):
    lhs = dT_T.pole(q)  # [∂T, T]_q
    rhs = -(q - 1) * T_T.pole(q - 1) if q > 1 else sp.S.Zero  # -(q-1)[T, T]_{q-1}
    
    # 简化并比较
    diff = sp.expand(lhs - rhs)
    
    print(f"q = {q}:")
    print(f"  [∂T, T]_{q} = {lhs}")
    print(f"  -(q-1)[T, T]_{{{q-1}}} = {rhs}")
    print(f"  差值 = {diff}")
    print(f"  验证: {'✓ 通过' if diff == 0 else '✗ 失败'}")
    print()

OPE[∂T, T]:
<< 5|| -2*c*One 3|| -4*T 2|| -∂T >>

OPE[T, T]:
<< 4|| c*One/2 2|| 2*T 1|| ∂T >>

验证 eq 3.3.1:
q = 1:
  [∂T, T]_1 = 0
  -(q-1)[T, T]_{0} = 0
  差值 = 0
  验证: ✓ 通过

q = 2:
  [∂T, T]_2 = -∂T
  -(q-1)[T, T]_{1} = -∂T
  差值 = 0
  验证: ✓ 通过

q = 3:
  [∂T, T]_3 = -4*T
  -(q-1)[T, T]_{2} = -4*T
  差值 = 0
  验证: ✓ 通过

q = 4:
  [∂T, T]_4 = 0
  -(q-1)[T, T]_{3} = 0
  差值 = 0
  验证: ✓ 通过

q = 5:
  [∂T, T]_5 = -2*c*One
  -(q-1)[T, T]_{4} = -2*c*One
  差值 = 0
  验证: ✓ 通过



### 13.1 验证 eq 3.3.1: [∂T, T]_q = -(q-1)[T, T]_{q-1}

我们已经定义了 T(z)T(w) 的 OPE：
$$T(z)T(w) \sim \frac{c/2}{(z-w)^4} + \frac{2T(w)}{(z-w)^2} + \frac{\partial T(w)}{z-w}$$

现在计算 $[\partial T, T]_q$ 并验证它是否等于 $-(q-1)[T, T]_{q-1}$。

## 13. 验证 Thielemans 论文中的导数算符 OPE 规则

根据 [Thielemans] 论文的 eq 3.3.1 和 3.3.2，导数算符的 OPE 应满足：

**eq 3.3.1**: $[\partial A, B]_q = -(q-1)[A, B]_{q-1}$

**eq 3.3.2**: $[A, \partial B]_q = (q-1)[A, B]_{q-1} + \partial[A, B]_q$

我们使用 Virasoro 代数来验证这些关系。